# Packages 

In [12]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
from math import *
from sklearn.metrics import r2_score
import seaborn as sns
import warnings
warnings.filterwarnings("ignore") 
%matplotlib inline
from tqdm import tqdm
import csv as csv

In [58]:
output_path = '/Users/lanauaa/Desktop/Viral_dynamics_project/Data_replication_immunotron/'
experiment_name = '20230306_MHV_HeLa_kinetics'
name_rawdata_file = '20230202_MHV_HeLa_sybr'
plate_number='1'

# Import Data 

In [33]:
#Convert txt to csv
convertion_to_csv = pd.read_csv('{}/{}/Data/{}.txt'.format(output_path,experiment_name,name_rawdata_file),sep='\t')
convertion_to_csv.drop(['Include','Name','Concentration','Standard','Status'], axis=1,inplace=True)
convertion_to_csv.to_csv('{}/{}/Data/{}.csv'.format(output_path,experiment_name,name_rawdata_file))

#Import data
PCR_data_raw = pd.read_csv('{}/{}/Data/{}.csv'.format(output_path,experiment_name,name_rawdata_file)).drop(['Unnamed: 0'],axis=1).rename(columns={'Pos':'Tag','Cp':'CT'})
                        
PCR_data = PCR_data_raw.dropna(axis=0, inplace=False)

#Define Titers and Timepoints
titers_list=[10,1,0.1,0.01,0.001,0.0001,0.00001,0]
timepoints_list=[24,20,16,12,8,5,4,3,2,1.5,1,0.5]

PCR_data

,,Color,Tag,HK,GOI,Timepoint,Titer,DeltaCT,Delta2CT,GeneExpression
,GeneType,,,,,,,,,
0,HK,255,A1,18.27,13.69,24.0,10.00000,-4.58,-19.315,6.522204e+05
12,HK,255,B1,18.56,13.10,24.0,10.00000,-5.46,-20.195,1.200330e+06
23,HK,255,C1,17.47,23.29,24.0,1.00000,5.82,-8.915,4.827057e+02
34,HK,255,D1,18.50,25.54,24.0,1.00000,7.04,-7.695,2.072172e+02
44,HK,255,E1,18.57,23.69,24.0,0.10000,5.12,-9.615,7.841577e+02
...,...,...,...,...,...,...,...,...,...,...
132,HK,255,L12,16.56,30.03,0.5,0.00010,13.47,1.815,2.842042e-01
144,HK,255,M12,15.84,40.00,0.5,0.00001,24.16,12.505,1.720362e-04
156,HK,255,N12,16.35,35.54,0.5,0.00001,19.19,7.535,5.391865e-03


In [60]:
#Convert txt to csv
convertion_to_csv = pd.read_csv('{}/{}/Data/{}.txt'.format(output_path,experiment_name,name_rawdata_file),sep='\t')
convertion_to_csv.drop(['Include','Name','Concentration','Standard','Status'], axis=1,inplace=True)
convertion_to_csv.to_csv('{}/{}/Data/{}.csv'.format(output_path,experiment_name,name_rawdata_file))

#Import data
PCR_data_raw = pd.read_csv('{}/{}/Data/{}.csv'.format(output_path,experiment_name,name_rawdata_file)).drop(['Unnamed: 0'],axis=1).rename(columns={'Pos':'Tag','Cp':'CT'})
PCR_data = PCR_data_raw.dropna(axis=0, inplace=False)              

#define HK and GOI
PCR_data['GeneType']=['HK' if int(x[1:])<13 else 'GOI' for x in PCR_data['Tag']]
PCR_data = PCR_data.set_index(['GeneType'],append=True)

sub_df=PCR_data.query('GeneType=="HK"').reset_index().drop(['level_0'],axis=1)
sub_df['HK']=sub_df['CT']
sub_2df=PCR_data.query('GeneType=="GOI"').reset_index().drop(['level_0'],axis=1)
sub_df['GOI']=sub_2df['CT']
PCR_data=sub_df.drop(['CT','GeneType'],axis=1)

#set timepoints and titers
df_list=[]

PCR_data['numbers']=[int(x[1:]) for x in PCR_data['Tag']]
numbers_list=PCR_data['numbers'].unique()
for i,time in enumerate(timepoints_list):
    i=i+1
    sub_df = PCR_data.query('numbers==@i')
    sub_df['Timepoint']=time
    df_list.append(sub_df)
PCR_data=pd.concat(df_list)

df2_list=[]
PCR_data['letters']=[x[0] for x in PCR_data['Tag']]
letters_list=PCR_data['letters'].unique()
for i,titer in enumerate(titers_list):
    j=letters_list[0+2*i]
    k=letters_list[1+2*i]
    sub_df = PCR_data.query('letters==@j')
    sub_df['Titer']= titer
    df2_list.append(sub_df)
    sub_df = PCR_data.query('letters==@k')
    sub_df['Titer']= titer
    df2_list.append(sub_df)

PCR_data=pd.concat(df2_list)


#delta2CT method
df3_list=[]
PCR_data['DeltaCT']=PCR_data['GOI']-PCR_data['HK']
for number in numbers_list:
    sub_df=PCR_data.query('numbers==@number')
    list=[sub_df.query('letters=="O"')['DeltaCT'],sub_df.query('letters=="P"')['DeltaCT']]
    delta=np.mean(list)
    sub_df['Delta2CT']=sub_df['DeltaCT']-delta
    df3_list.append(sub_df)
PCR_data=pd.concat(df3_list).drop(['numbers','letters'],axis=1)
PCR_data["GeneExpression"]=2**(-PCR_data['Delta2CT'])

#Mean of technical duplicate/triplicate 
list4_df=[]
for time in timepoints_list:
    sub_df=PCR_data.query('Timepoint==@time')
    for titer in titers_list:
        sub_sub_df = sub_df.query('Titer==@titer')
        mean_expression = np.mean(sub_sub_df['GeneExpression'])
        sub_sub_df['GeneExpression']=mean_expression
        mean_df=sub_sub_df.drop_duplicates('GeneExpression').drop(['Tag','HK','GOI','DeltaCT','Delta2CT','Color'],axis=1)
        list4_df.append(mean_df)
    PCR_data_mean = pd.concat(list4_df)

PCR_data_mean.to_csv('{}/{}/Data/Analyzed_PCR_Data_{}.csv'.format(output_path,experiment_name,plate_number))

PCR_data_mean

,Timepoint,Titer,GeneExpression
0,24.0,10.00000,926275.331554
23,24.0,1.00000,344.961445
44,24.0,0.10000,408.246070
65,24.0,0.01000,8.088467
87,24.0,0.00100,5.483056
...,...,...,...
76,0.5,0.01000,16.598754
98,0.5,0.00100,0.158430
121,0.5,0.00010,0.233925
144,0.5,0.00001,0.002782
